In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################
start_time = time.time();

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
job_array=False;index_adjust=0
ocean_fraction=2/8

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# dx = 1km; Np = 50M; Nz = 95
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc') #***
res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
############################################################################################
times=data['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1)

In [ ]:
# #JOB ARRAY SETUP
# job_array=True
# if job_array==True:

#     num_jobs=60 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
#     total_elements=len(parcel['xh']) #total num of variables

#     if num_jobs >= total_elements:
#         raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
#     job_range = total_elements // num_jobs  # Base size for each chunk
#     remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
#     # Function to compute the start and end for each job_id
#     def get_job_range(job_id):
#         job_id-=1
#         # Add one extra element to the first 'remaining' chunks
#         start_job = job_id * job_range + min(job_id, remaining)
#         end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
#         if job_id == num_jobs - 1: 
#             end_job = total_elements #- 1
#         return start_job, end_job
#     # def job_testing():
#     #     #TESTING
#     #     start=[];end=[]
#     #     for job_id in range(1,num_jobs+1):
#     #         start_job, end_job = get_job_range(job_id)
#     #         print(start_job,end_job)
#     #         start.append(start_job)
#     #         end.append(end_job)
#     #     print(np.all(start!=end))
#     #     print(len(np.unique(start))==len(start))
#     #     print(len(np.unique(end))==len(end))
#     # job_testing()
    
#     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
#     if job_id==0: job_id=1
#     start_job, end_job = get_job_range(job_id)
#     job_adjust=start_job
#     print(f'start_job = {start_job}, end_job = {end_job}')


In [ ]:
###################################################################################################################################
#GENERAL FUNCTIONS

In [ ]:
#Get Data Functions
def get_2dtime_data(data,varname,tlev,zlev):
    cloud_var=data[varname].isel(time=tlev,zh=zlev).values
    return cloud_var
def get_3dtime_data(data,varname,tlev):
    cloud_var=data[varname].isel(time=tlev).values
    return cloud_var

def get_conv(t):
    import h5py
    # print('calculating convergence and taking mean')
    dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    # file_path = dir2 + 'Variable_Calculation/' + 'Convergence' + f'_{res}_{t_res}' + '.h5'
    file_path = dir2 + 'Variable_Calculation/' + 'Convergence' + f'_{res}_{t_res}.h5'
    with h5py.File(file_path, 'r') as f:
        Conv = f['conv'][t]
    return Conv

In [ ]:
#TIME 00:00:00 Function
#Gets the realtime for the current timestep
def get_time(t):
    # init_day,init_hour,init_min=0,0,0
    init_day,init_hour,init_min=0,6,0
    times=data['time'].values/(1e9 * 60); time_inc=times.astype(int)[1]-times.astype(int)[0]
    current_min=init_hour*60+init_min+time_inc*t;
    
    days = init_day + (current_min // (24 * 60))
    
    remain_min = (init_min+time_inc*t) % (24 * 60); 
    hours = (init_hour + (remain_min // 60)) % 24
    mins = remain_min % 60

    ##############################################
    days=str(days);hours=str(hours);mins=str(mins)
    if len(days)==1:days='0'+days
    if len(hours)==1:hours='0'+hours
    if len(mins)==1:mins='0'+mins
    ##############################################

    combo=days+":"+hours+":"+mins
    return days,hours,mins

In [ ]:
###################################################################################################################################
#ALGORITHM FUNCTIONS

In [ ]:
#Function for taking x and y derivatives (Gradient)
def cd2d(f,dx,dy): #size not compatible, cant calculate adjacent gradient
    ddx = (
            f[:,:, 1:  ]
            -
            f[:,:, 0:-1]
        ) / (
        2 * dx
    )
    
    ddy = (
        f[:,1:, :]
        -
        f[:,0:-1, :]
    ) / (
        2 * dy
    )
    
    return ddx, ddy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr; import time as time
start_time=time.time()
folder_path = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/trackout/CL_tracking_plots/' #*** plot image output foldername
import os; os.makedirs(folder_path, exist_ok=True)
####################################################################################################################################
def user_deletion():
    #reads user input for timesteps to forget as instructed by teh user in the usersbz.txt file
    filename='usersbz_1km.txt' #*** timestep deletion filename
    # filename='usersbz_250m.txt' #*** timestep deletion filename
    a,b,c,d=[],[],[],[]
    try:
        with open(folder_path+filename, 'r') as file: #*switch to lbz for land breezes 
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace
                if ',' in line:
                    if '-' not in line: #single case correction
                        # For rows with commas, split and append to a, b, and c lists
                        values = line.split(',')
                        a.append(int(values[0]))
                        b.append(int(values[1]))
                        c.append(int(values[2]))
                    elif '-' in line: #multiple case correction
                        values = line.split(',')   
                        start, end = map(int, values[0].split('-'))
                        a.extend(range(start, end + 1))
                        b=b+[int(values[1])]*((end-start)+1)
                        c=c+[int(values[2])]*((end-start)+1)
                elif '-' in line and ',' not in line: #ignore these
                    # For rows with hyphen, append the range to d list
                    start, end = map(int, line.split('-'))
                    d.extend(range(start, end + 1))
    
    except FileNotFoundError:
        print(f'The file {filename} does not exist. Add if needed.')
    forget = d #needed for single and all max algorithm
    return forget
####################################################################################################################################

In [ ]:
def find_SBZ_xmaxs():
    # Define the directory and file path
    dir2 = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    file_path = dir2 + 'Variable_Calculation/' + 'Convergence' + f'_{res}_{t_res}.h5'
    
    # Open the HDF5 file in read mode
    with h5py.File(file_path, 'r') as f:
        # Access the 'conv' dataset
        conv_dataset = f['conv']
        
        # Define the vertical level you are interested in
        zlev = 4
        
        # Initialize a list to store the xmaxs for each time step
        xmaxs_list = []

        # Loop over each time step (axis=0 corresponds to time)
        for t in range(conv_dataset.shape[0]):  # conv_dataset.shape[0] is the time dimension size
            # Read the relevant slice for this time step and vertical level
            Conv_t_zlev = conv_dataset[t, zlev, :, :]  # Shape should be (y_size, x_size)
            
            # Calculate the mean across the y-axis
            Conv_ymean = np.mean(Conv_t_zlev, axis=0)  # Mean across the y-axis
            
            # Find the index of the maximum value along the x-axis
            xmax = np.argmax(Conv_ymean)
            
            # Append the result for this time step
            xmaxs_list.append(xmax)
    
    # Convert the list of xmaxs to a numpy array (optional)
    xmaxs = np.array(xmaxs_list)

    return xmaxs #returns SBZ x location for each timestep
SBZ_MAXS=find_SBZ_xmaxs()
print('done')

In [ ]:
#Finds all local maximums (from Calculus) along each y level for a specific z level (~0.28km in this case)
def find_local_maxes(conv_z,t,yind,conv_thresh,ONLY_SBZ):
    yconv=conv_z[yind,:]
    xf=data['xf'].values.astype(int);
    kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1) #finds how many x grids is 1 km
    dx=np.round(data['xf'][1]-data['xf'][0],1).item()
    
    #takes dconv/dx
    f=yconv
    ddx = (
            f[1:  ]
            -
            f[0:-1]
        ) / (
        2 * dx
    )

    if np.any(np.isin(forget,t)): 
        local_maxes=np.full_like(local_maxes,np.nan,dtype=float); #forget timestep if in forget variable
    else:
        #finds local max where dconv/dx sign changes
        signs = np.sign(ddx)
        signs_diff=np.diff(signs)
        local_maxes=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here (it corrects the location of the derivative)
        local_maxes=local_maxes[np.where(yconv[local_maxes]>conv_thresh)] #check if convergence is greater than convergence threshold (0.9s-1)
        local_maxes=local_maxes[(local_maxes>50*kms)&(local_maxes<len(xf)-50*kms)] #removes maxes that are with 50 km of y boundary
        # local_maxes=local_maxes[local_maxes>int(len(xf)*ocean_fraction)] #restricts to right land side
        if ONLY_SBZ==True:
            local_maxes=local_maxes[(local_maxes>=SBZ_MAXS[t]-10*kms)&(local_maxes<=SBZ_MAXS[t]+10*kms)] #removes maxes that are with 50 km of y boundary
            # print(SBZ_MAXS[t]) #TESTING
        # ################################################################################
        # #second round maxes (not 100% necessary, only if missing many convergence maximums that are visually there)
        # yconv2=yconv.copy()
        # yconv2[local_maxes]=0
        # #takes dconv/dx
        # f=yconv2
        # ddx = (
        #         f[1:  ]
        #         -
        #         f[0:-1]
        #     ) / (
        #     2 * dx
        # )
        # signs = np.sign(ddx)
        # signs_diff=np.diff(signs)
        # local_maxes2=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here
        # local_maxes2=local_maxes2[np.where(yconv2[local_maxes2]>conv_thresh)] #remove local maxes less than zero
        # local_maxes2=local_maxes2[(local_maxes2>50*kms)&(local_maxes2<len(xf)-50*kms)] #removes maxes that are with 50 km of y boundary
        # local_maxes2=local_maxes2[local_maxes2>int(len(xf)/2)] #restricts to right land side
        # local_maxes=np.concatenate((local_maxes,local_maxes2))
        # ################################################################################
    return ddx,local_maxes

In [ ]:
###################################################################################################################################
#Calculation Run

In [ ]:
#Find_Local_Maxes Function
#(1) At a single time and z level, runs through each y-level
#(2) At each y-level, takes the x-derivative
#(3) Take sign(x_derivative)
#(4) Take diff(x_derivative)
#(5) Max is located one index to the right of where derivative changes from positive to negative or diff is +1
#[(6) Optional: the algorithm can run a second time over the leftover maxes after removing previous maxes from temporary variable]

In [ ]:
#SBZ Convergence Line Search Algorithm (levels are seperate) (python version 3.10.9) (All Max Algorithm)
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr; import time as time
start_time=time.time()
####################################################################################################################################
forget=user_deletion() #get forget variable for user specified timesteps to forget
####################################################################################################################################

def layermax(t,ONLY_SBZ): #finds max convergence along y for multiple z location (5 is good)
    num_zlevs=16 #5
    maxconv_x=np.full((num_zlevs+1,len(data['yh']),len(data['xh'])), -1, dtype=int)
    #running again for all levels 
    for zlev in range(0,num_zlevs+1):
        #Taking Convergence of current timestep
        dx=np.round(data['xf'][1]-data['xf'][0],2).item();dy=dx #gets the dx,dy in meters (0.25m in this case)
        conv=get_conv(t)
    
        conv_z=conv[zlev,:,:] #current z level for convergence

        for yind in range(0,len(data['yh'])): #plot maximums for each row
            #finds all local maxes
            if res=='1km':
                [ddx,local_maxes]=find_local_maxes(conv_z,t,yind,0.9/1000,ONLY_SBZ) #finds all local maxes with threshold 1km:0.9 250m:3.0
            elif res=='250m':
                [ddx,local_maxes]=find_local_maxes(conv_zt,yind,3.0/1000,ONLY_SBZ) #finds all local maxes with threshold 1km:0.9 250m:3.0
            
            #storing data
            maxconv_x[zlev,yind,local_maxes] = local_maxes
    return maxconv_x

In [ ]:
ONLY_SBZ=False
# ONLY_SBZ=True
#RUNNING ALGORITHM
def RunAlgorithm():

    maxconv_x=layermax(0,ONLY_SBZ)
    ds1= xr.Dataset({'maxconv_x': (['z','y','x'], maxconv_x)})
    for t in range(1,len(data['time'])): #starts from timestep 1 to end
        if np.mod(t,10)==0: print(f'current time step: {t}/{len(data["time"])}')
            
        maxconv_x=layermax(t,ONLY_SBZ)
        ds2= xr.Dataset({'maxconv_x': (['z','y','x'], maxconv_x)})
        ds1=xr.concat([ds1, ds2], dim='time')
    return ds1
RunAlgorithm()
    

#SAVING DATA
def SaveData(ds1):
    print('saving')
    output_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/trackout/CL_tracking_plots/'
    if ONLY_SBZ==False:
        output_folder += 'ALL_CLS/'
    elif ONLY_SBZ==True:
        output_folder += 'ONLY_SBZS'
    
    # if job_array==True: #USING JOB ARRAY (NO NEED CURRENTLY)
    #     if ONLY_SBZ==False:
    #         out_name=output_folder2+f'whereCL_{res}_{t_res}_ALL_CLS_{job_id}.nc'
    #     elif ONLY_SBZ==True:
    #         out_name=output_folder2+f'whereCL_{res}_{t_res}_ONLY_SBZS_{job_id}.nc'
    #     ds1.to_netcdf(out_name)  
    # elif job_array==False: #NOT USING JOB ARRAY
    output_folder2 = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/'
    if ONLY_SBZ==False:
        out_name=output_folder2+f'whereCL_{res}_{t_res}_ALL_CLS.nc'
        ds1.to_netcdf(out_name) 
    elif ONLY_SBZ==True:
        out_name=output_folder2+f'whereCL_{res}_{t_res}_ONLY_SBZS.nc'
        ds1.to_netcdf(out_name) 
    print('done')
SaveData(ds1)

end_time = time.time(); elapsed_time = end_time - start_time; print(f"Total Elapsed Time: {elapsed_time} seconds")
#Nt=650,Nx=500,Ny=200 ==> * minutes

In [ ]:
###################################################################################################################################
#PLOTTING
plotting=False #KEEP FALSE WHEN RUNNING
# plotting=True

In [ ]:
folder_path = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/trackout/CL_tracking_plots/' #plot image output foldername

load_dir = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/'

open_name = load_dir+f'whereCL_{res}_{t_res}_ALL_CLS.nc'
whereCL_ALL_CLS=xr.open_dataset(open_name)['maxconv_x'].data

# open_name = load_dir+f'whereCL_{res}_{t_res}_{Np_str}_ONLY_SBZS.nc'
# whereCL_ONLY_SBZS=xr.open_dataset(open_name)['maxconv_x'].data

def CL_plotting(t,ONLY_SBZ,SAVING):
    if np.mod(t,10)==0: print(f'current time step: {t}/{len(data["time"])}')

    #Taking Convergence of current timestep
    dx=np.round(data['xf'][1]-data['xf'][0],2).item();dy=dx #gets the dx,dy in meters (0.25m in this case)
    conv=get_conv(t)

    #Plotting horizontal layer at ~0.28 km 
    zlev=3 #(zf level ~0.28 km)
    channel_aspect_ratio = 5
    figwidth=20
    plt.figure(figsize=(figwidth, figwidth/channel_aspect_ratio)) 
    contour=plt.contourf(conv[zlev]*1000)#,levels=np.arange(-1.5,1.5,.01),vmin=vmin,vmax=vmax)
    
    colorbar = plt.colorbar(contour,label=f'{dx*1000:.0f}'+r'$\ s^{-1}$')# ,pad=pad)
    #####################################################################
    #Gets time for Plot Label
    [days,hours,mins]=get_time(t+index_adjust)
    value=data['zf'][zlev].values; 
    plt.title(f'Convergence at t={t+index_adjust}={days}:{hours}:{mins}, z={value:.2f} km')
    plt.xlabel('x (km)')
    plt.ylabel('y (km)')
    #####################################################################

    # plt.contourf(data['pwat'].isel(time=t),alpha=0.3,cmap='Reds') #rain, prate, pwat #TESTING

    #ADDING TRACKED LOCATION SCATTER
    #for each ylevel, apply FindLocalMaxes Function
    zlev=3;conv_z=conv[zlev,:,:] #only look at 3rd level of convergence field
    maxconv_x=np.array([],dtype='int'); 
    for yind in range(0,len(data['yh'])): #plot maximums for each row
        if ONLY_SBZ==False:
            local_maxes=whereCL_ALL_CLS[t,3,yind]
            local_maxes=local_maxes[local_maxes != -1]
        elif ONLY_SBZ==True:
            local_maxes=whereCL_ONLY_SBZS[t,3,yind]
            local_maxes=local_maxes[local_maxes != -1]
        plt.scatter(local_maxes,np.array([yind]*len(local_maxes)),color='red',s=1)

    # SAVING PLOT
    if SAVING==True:
        if ONLY_SBZ==False:
            plt.savefig(os.path.join(folder_path, f"ALL_CLS/plot_{t+index_adjust}.png"),dpi=72) #save the plot
        if ONLY_SBZ==True:
            plt.savefig(os.path.join(folder_path, f"ONLY_SBZS/plot_{t+index_adjust}.png"),dpi=72) #save the plot
        # print('finished saving plot') #TESTING
        plt.close()

In [ ]:
if plotting==True:
    #TESTING INDIVIDUAL PLOTS
    #########################
    t=200
    t+=20
    CL_plotting(t=t,ONLY_SBZ=False,SAVING=False)

In [ ]:
if plotting==True:
    #OUTPUTTING ALL PLOTS
    #####################
    for t in np.arange(len(data['time'])):
        CL_plotting(t=t,ONLY_SBZ=False,SAVING=True)
        # CL_plotting(t=t,ONLY_SBZ=True,SAVING=True)

In [ ]:
if plotting==True:
    #MAKE A GIF OF ALL TRACKED IMAGES
    #####################
    from PIL import Image
    import os
    
    ONLY_SBZ=False
    # ONLY_SBZ=True
    # Folder containing the pictures
    if ONLY_SBZ==False:
        input_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/trackout/CL_tracking_plots/ALL_CLS/'
    if ONLY_SBZ==True:
        input_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/trackout/CL_tracking_plots/ONLY_SBZS/'
    
    image_filenames = sorted([filename for filename in os.listdir(input_folder) if filename.endswith((".jpg", ".png"))], 
                             key=lambda x: int(x.split('_')[-1].split('.')[0]))
    
    # List to store image objects
    images = []
    
    # Iterate over all files in the folder
    for filename in image_filenames:
        # Check if the file is an image
        if (filename.endswith(".jpg") or filename.endswith(".png")):
            # Open the image and append it to the list
            images.append(Image.open(os.path.join(input_folder, filename)))
    
    # Save images as a GIF
    if ONLY_SBZ==False:
        output_path = input_folder+f'CL_plots_{res}_{t_res}_ALL_CLS.gif'
    if ONLY_SBZ==True:
        output_path = input_folder+f'CL_plots_{res}_{t_res}_ONLY_SBZS.gif'
    images[0].save(output_path,
                   save_all=True,
                   append_images=images[1:],
                   duration=100,  # Specify duration for each frame in milliseconds
                   loop=0)

In [ ]:
if plotting==True:
    def convert_gif_to_mp4(input_file, output_file, fps,speed,bitrate='750k'):
        from moviepy.editor import VideoFileClip, vfx
        # Load the GIF file
        gif_clip = VideoFileClip(input_file)
    
        # Set the desired framerate if provided
        if fps:
            gif_clip = gif_clip.set_fps(fps)
        if speed != 1.0:
            gif_clip = gif_clip.fx(vfx.speedx, speed)
    
        # Write the GIF as an MP4 file
        gif_clip.write_videofile(output_file, codec="libx264",bitrate=bitrate)
    
    ONLY_SBZ=False
    # ONLY_SBZ=True
    input_folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/trackout/CL_tracking_plots/'
    if ONLY_SBZ==False:
        input_folder += 'ALL_CLS/'
    elif ONLY_SBZ==True:
        input_folder += 'ONLY_SBZS/'
    input_file = input_folder+f'CL_plots_{res}_{t_res}_ALL_CLS.gif'
    
    if ONLY_SBZ==False:
        output_file = f'/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_plots_{res}_{t_res}_ALL_CLS.mp4'
    if ONLY_SBZ==True:
            output_file = f'/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/CL_plots_{res}_{t_res}_ONLY_SBZS.mp4'
        
    convert_gif_to_mp4(input_file, output_file, speed=0.25,fps=None)